In [1]:
# 9.3 特征提取
# 4.ArcFace

In [2]:
class ArcMarignProduct(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        # 初始化权重
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)
        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m)*m
    
    def forward(self, input, label):
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine*self.cos_m - sine*self.sin_m
        if self.easy_margine:
            phi = torch.where(cosine>0, phi, cosine)
        else:
            phi = torch.where(cosine>self.th, phi, cosine-self.mm)
        # 将cos（\theta+m）更新到tensor相应的位置
        one_hot = torch.zeros(cosine.size(), device='cuda')
        # scatter_(input, dim, index, src)将src中数据根据index中的索引按照dim的方向填进input中
        one_hot.scatter_(1, label.view(-1,1).long(), 1)
        output = (one_hot*phi) + ((1.0-one_hot)*cosine)
        output *= self.s
        return output

SyntaxError: unexpected EOF while parsing (<ipython-input-2-2b5ea117f583>, line 1)

In [3]:
# 9.5 PyTorch实现人脸检测与识别
# 9.5.1 验证检测代码
# 1.查看图像及检测效果
from PIL import Image
from face_dect_recong.align.detector import detect_faces
from face_detect_recong.align.visualization_utils import show_results
%matplotlib inline
img = Image.open('./data/face.jpg')
bounding_boxes, landmarks = detect_faces(img)
show_results(img, bounding_boxes, landmarks)

ModuleNotFoundError: No module named 'face_dect_recong'

In [ ]:
# 9.5.2 检测图像
# 1. 对图像进行检测
%run face_dect_recong/align/face_align.py -source_root './data/' -dest_root './data/' -crop_size 128

In [ ]:
# 9.5.3 检测后进行预处理
# 删除检测后头像小于4张的一些人
%run face_dect_recong/balance/remove_lowshot.py -root './data/' -min_num 4

In [ ]:
# 9.5.4 查看经检测后的图像
# 查看图像检测结果
import matplotlib.pyplot as plt
from matplotlib.image import imread
%matplotlib inline
img = imread('./data/face.jpg')
plt.imshow(img)
plt.show

In [ ]:
# 9.5.5 人脸识别
# 1.定义下载预训练模型url及device
model_urls = {
    'resnet18': 'https://dowload.PyTorch.org/models/resnet18-5c106cde.pth'
    }
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# 2.调用其他模块的主程序
opt = Config()
model = renet_face18(opt.use_se)
# 采用多GPU的数据并行处理机制
model = DataParallel(model)
# 装载与训练模型
model.load_state_dict(torch.load(opt.test_model_path))
model.to(device)
identity_list = get_lfw_list(opt.lfw_test_list)
img_paths = [os.path.join(opt.lfw_root, each) for each in identity_list]
model.eval()
lfw_test(model, img_paths, identity_list, opt.lfw_test_list, opt.test_batch_size)

NameError: name 'Config' is not defined